In [1]:
pip install python-chess


Note: you may need to restart the kernel to use updated packages.


In [3]:
import chess

# 1) Material weights (centipawns) for opening and endgame, indexed by piece_type−1
_material = {
    'opening': [0, 124, 781, 825, 1276, 2538],   # PAWN, KNIGHT, BISHOP, ROOK, QUEEN, KING
    'endgame': [0, 206, 854, 915, 1380, 2682],
}
_opening_phase = 15258
_endgame_phase = 3915

# 2) Piece-Square Tables (flattened 8×8 → 64 entries) for each piece and phase
_pst = {
    chess.PAWN: {
        'opening': [
             0,   0,   0,   0,   0,   0,   0,   0,
            -7,   7,  -3, -13,   5, -16,  10,  -8,
             5, -12,  -7,  22,  -8,  -5, -15,  -8,
            13,   0, -13,   1,  11,  -2, -13,   5,
            -4, -23,   6,  20,  40,  17,   4,  -8,
            -9, -15,  11,  15,  32,  22,   5, -22,
             3,   3,  10,  19,  16,  19,   7,  -5,
             0,   0,   0,   0,   0,   0,   0,   0,
        ],
        'endgame': [
             0,   0,   0,   0,   0,   0,   0,   0,
             0, -11,  12,  21,  25,  19,   4,   7,
            28,  20,  21,  28,  30,   7,   6,  13,
            10,   5,   4,  -5,  -5,  -5,  14,   9,
             6,  -2,  -8,  -4, -13, -12, -10,  -9,
           -10, -10, -10,   4,   4,   3,  -6,  -4,
           -10,  -6,  10,   0,  14,   7,  -5, -19,
             0,   0,   0,   0,   0,   0,   0,   0,
        ],
    },
    chess.KNIGHT: {
        'opening': [
          -201, -83, -56, -26, -26, -56, -83, -201,
           -67, -27,   4,  37,  37,   4, -27,  -67,
            -9,  22,  58,  53,  53,  58,  22,   -9,
           -34,  13,  44,  51,  51,  44,  13,  -34,
           -35,   8,  40,  49,  49,  40,   8,  -35,
           -61, -17,   6,  12,  12,   6, -17,  -61,
           -77, -41, -27, -15, -15, -27, -41,  -77,
          -175, -92, -74, -73, -73, -74, -92, -175,
        ],
        'endgame': [
          -100, -88, -56, -17, -17, -56, -88, -100,
           -69, -50, -51,  12,  12, -51, -50,  -69,
           -51, -44, -16,  17,  17, -16, -44,  -51,
           -45, -16,   9,  39,  39,   9, -16,  -45,
           -35,  -2,  13,  28,  28,  13,  -2,  -35,
           -40, -27,  -8,  29,  29,  -8, -27,  -40,
           -67, -54, -18,   8,   8, -18, -54,  -67,
           -96, -65, -49, -21, -21, -49, -65,  -96,
        ],
    },
    chess.BISHOP: {
        'opening': [
           -48,   1, -14, -23, -23, -14,   1, -48,
           -17, -14,   5,   0,   0,   5, -14, -17,
           -16,   6,   1,  11,  11,   1,   6, -16,
           -12,  29,  22,  31,  31,  22,  29, -12,
            -5,  11,  25,  39,  39,  25,  11,  -5,
            -7,  21,  -5,  17,  17,  -5,  21,  -7,
           -15,   8,  19,   4,   4,  19,   8, -15,
           -53,  -5,  -8, -23, -23,  -8,  -5, -53,
        ],
        'endgame': [
           -46, -42, -37, -24, -24, -37, -42, -46,
           -31, -20,  -1,   1,   1,  -1, -20, -31,
           -30,   6,   4,   6,   6,   4,   6, -30,
           -17,  -1, -14,  15,  15, -14,  -1, -17,
           -20,  -6,   0,  17,  17,   0,  -6, -20,
           -16,  -1,  -2,  10,  10,  -2,  -1, -16,
           -37, -13, -17,   1,   1, -17, -13, -37,
           -57, -30, -37, -12, -12, -37, -30, -57,
        ],
    },
    chess.ROOK: {
        'opening': [
           -17, -19,  -1,   9,   9,  -1, -19, -17,
            -2,  12,  16,  18,  18,  16,  12,  -2,
           -22,  -2,   6,  12,  12,   6,  -2, -22,
           -27, -15,  -4,   3,   3,  -4, -15, -27,
           -13,  -5,  -4,  -6,  -6,  -4,  -5, -13,
           -25, -11,  -1,   3,   3,  -1, -11, -25,
           -21, -13,  -8,   6,   6,  -8, -13, -21,
           -31, -20, -14,  -5,  -5, -14, -20, -31,
        ],
        'endgame': [
             18,   0,  19,  13,  13,  19,   0,  18,
              4,   5,  20,  -5,  -5,  20,   5,   4,
              6,   1,  -7,  10,  10,  -7,   1,   6,
             -5,   8,   7,  -6,  -6,   7,   8,  -5,
             -6,   1,  -9,   7,   7,  -9,   1,  -6,
              6,  -8,  -2,  -6,  -6,  -2,  -8,   6,
            -12,  -9,  -1,  -2,  -2,  -1,  -9, -12,
             -9, -13, -10,  -9,  -9, -10, -13,  -9,
        ],
    },
    chess.QUEEN: {
        'opening': [
            -2,  -2,   1,  -2,  -2,   1,  -2,  -2,
            -5,   6,  10,   8,   8,  10,   6,  -5,
            -4,  10,   6,   8,   8,   6,  10,  -4,
             0,  14,  12,   5,   5,  12,  14,   0,
             4,   5,   9,   8,   8,   9,   5,   4,
            -3,   6,  13,   7,   7,  13,   6,  -3,
            -3,   5,   8,  12,  12,   8,   5,  -3,
             3,  -5,  -5,   4,   4,  -5,  -5,   3,
        ],
        'endgame': [
           -75, -52, -43, -36, -36, -43, -52, -75,
           -50, -27, -24,  -8,  -8, -24, -27, -50,
           -38, -18, -12,   1,   1, -12, -18, -38,
           -29,  -6,   9,  21,  21,   9,  -6, -29,
           -23,  -3,  13,  24,  24,  13,  -3, -23,
           -39, -18,  -9,   3,   3,  -9, -18, -39,
           -55, -31, -22,  -4,  -4, -22, -31, -55,
           -69, -57, -47,  26,  26, -47, -57, -69,
        ],
    },
    chess.KING: {
        'opening': [
             59,  89,  45,  -1,  -1,  45,  89,  59,
             88, 120,  65,  33,  33,  65, 120,  88,
            123, 145,  81,  31,  31,  81, 145, 123,
            154, 179, 105,  70,  70, 105, 179, 154,
            164, 190, 138,  98,  98, 138, 190, 164,
            195, 258, 169, 120, 120, 169, 258, 195,
            278, 303, 234, 179, 179, 234, 303, 278,
            271, 327, 271, 198, 198, 271, 327, 271,
        ],
        'endgame': [
             11,  59,  73,  78,  78,  73,  59,  11,
             47, 121, 116, 131, 131, 116, 121,  47,
             92, 172, 184, 191, 191, 184, 172,  92,
             96, 166, 199, 199, 199, 199, 166,  96,
            103, 156, 172, 172, 172, 172, 156, 103,
             88, 130, 169, 175, 175, 169, 130,  88,
             53, 100, 133, 135, 135, 133, 100,  53,
              1,  45,  85,  76,  76,  85,  45,   1,
        ],
    },
}

# --- Additional evaluation parameters
BISHOP_PAIR_BONUS = 50
PASSED_PAWN_BONUS = 20
ISOLATED_PAWN_PENALTY = 10
ROOK_OPEN_FILE_BONUS = 20
MOBILITY_BONUS = 1  # per legal move difference

# --- Helper functions

def is_material_draw(board: chess.Board) -> bool:
    return board.is_insufficient_material()

# Pawn structure: passed and isolated pawns
def pawn_structure_score(board: chess.Board, color: bool) -> int:
    score = 0
    pawns = board.pieces(chess.PAWN, color)
    for sq in pawns:
        file = chess.square_file(sq)
        rank = chess.square_rank(sq)
        # isolated: no friendly pawns on adjacent files
        adj_files = [f for f in (file-1, file+1) if 0 <= f < 8]
        if not any(chess.square(f, r) in pawns for f in adj_files for r in range(8)):
            score -= ISOLATED_PAWN_PENALTY
        # passed: no enemy pawns ahead on same or adjacent files
        enemy_pawns = board.pieces(chess.PAWN, not color)
        direction = 1 if color else -1
        passed = True
        for f in ([file] + adj_files):
            for r in range(rank+direction, 8 if color else -1, direction):
                if chess.square(f, r) in enemy_pawns:
                    passed = False
        if passed:
            score += PASSED_PAWN_BONUS * (rank+1)
    return score

# Bishop pair bonus
def bishop_pair_score(board: chess.Board, color: bool) -> int:
    return BISHOP_PAIR_BONUS if len(board.pieces(chess.BISHOP, color)) >= 2 else 0

# Rook on open file bonus

def rook_open_file_score(board: chess.Board, color: bool) -> int:
    score = 0
    pawns = board.pieces(chess.PAWN, True) | board.pieces(chess.PAWN, False)
    for sq in board.pieces(chess.ROOK, color):
        file = chess.square_file(sq)
        if not any(chess.square(file, r) in pawns for r in range(8)):
            score += ROOK_OPEN_FILE_BONUS
    return score

# Mobility score: difference in legal moves

def mobility_score(board: chess.Board) -> int:
    white_moves = len(list(board.legal_moves)) if board.turn else len(list(board.copy(stack=False).legal_moves))
    board.turn = not board.turn
    black_moves = len(list(board.legal_moves))
    board.turn = not board.turn
    return (white_moves - black_moves) * MOBILITY_BONUS

# Quadratic interpolation weight
def interpolation_weight(gp: int) -> float:
    t = (gp - _endgame_phase) / (_opening_phase - _endgame_phase)
    t = max(0.0, min(1.0, t))
    return t*t  # quadratic easing into endgame

# Main improved eval function

def improved_stockfish_eval(board: chess.Board) -> int:
    if is_material_draw(board):
        return 0

    # 1) Game phase score using opening material
    gp = 0
    for pt in range(1, 6):  # 1=Pawn..5=Queen
        count = len(board.pieces(pt, True)) + len(board.pieces(pt, False))
        gp += count * _material['opening'][pt-1]

    # 2) Scores for opening and endgame
    score_o = score_e = 0
    for sq, piece in board.piece_map().items():
        pt = piece.piece_type
        color = 1 if piece.color else -1
        # material
        score_o += _material['opening'][pt-1] * color
        score_e += _material['endgame'][pt-1] * color
        # positional PST
        if pt in _pst:
            sq_eff = chess.square_mirror(sq) if not piece.color else sq
            score_o += _pst[pt]['opening'][sq_eff] * color
            score_e += _pst[pt]['endgame'][sq_eff] * color

    # 3) Structural & miscellanea
    for color in [True, False]:
        mult = 1 if color else -1
        score_o += bishop_pair_score(board, color) * mult
        score_e += bishop_pair_score(board, color) * mult
        score_o += pawn_structure_score(board, color) * mult
        score_e += pawn_structure_score(board, color) * mult
        score_o += rook_open_file_score(board, color) * mult
        score_e += rook_open_file_score(board, color) * mult

    # 4) Interpolate
    w_eg = interpolation_weight(gp)
    raw_score = score_e * w_eg + score_o * (1 - w_eg)

    # 5) Mobility & castling safety bonus
    raw_score += mobility_score(board)

    return int(round(raw_score))

# Example usage
if __name__ == "__main__":
    import random
    # generate and evaluate 5 random positions
    board = chess.Board()
    for i in range(5):
        board.reset()
        for _ in range(random.randint(2, 15)):
            if board.is_game_over(): break
            board.push(random.choice(list(board.legal_moves)))
        print(f"Position {i+1} FEN: {board.fen()}")
        print(board)
        print("Eval:", improved_stockfish_eval(board))
        print('-'*40)


Position 1 FEN: rnbqkbnr/pp1ppppp/8/2p5/8/8/PPPPPPPP/RNBQKBNR b KQkq - 1 2
r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Eval: 12
----------------------------------------
Position 2 FEN: r1bqkb1r/pppppp1p/8/6P1/4n3/1PnP4/P1PNP1PP/R1BQKBNR w KQkq - 3 8
r . b q k b . r
p p p p p p . p
. . . . . . . .
. . . . . . P .
. . . . n . . .
. P n P . . . .
P . P N P . P P
R . B Q K B N R
Eval: -64
----------------------------------------
Position 3 FEN: rnbqkbnr/pppppp1p/8/8/5Pp1/P6N/1PPPP1PP/RNBQKB1R b KQkq - 0 3
r n b q k b n r
p p p p p p . p
. . . . . . . .
. . . . . . . .
. . . . . P p .
P . . . . . . N
. P P P P . P P
R N B Q K B . R
Eval: 98
----------------------------------------
Position 4 FEN: rnb2bnr/pp1k1ppp/3pp3/2p5/4P2q/1P3PPP/P1PP4/RNBQKBNR w KQ - 1 6
r n b . . b n r
p p . k . p p p
. . . p p . . .
. . p . . . . .
. . . . P . . q
. P . . . P P P
P . P P . . . .
R N B Q K B N R
Eval: -88
-------------